In [1]:
import pickle 
import sys
from pprint import pprint

sys.path.append('../')

from methods.SimilarityTests import SimilarityTests
from methods.LMEO import LMEO

st = SimilarityTests()

### Collecting data for the best UK Universities. 

In [6]:
## Geting the top1 prompt
all_prompt_dict = st.get_prompt_dict()
print(all_prompt_dict['pairwise'])
print(all_prompt_dict)


Which hotel do you recommend more: {} or {}. Simply write the name of the hotel and nothing else.
{'pairwise': 'Which hotel do you recommend more: {} or {}. Simply write the name of the hotel and nothing else.', 'topk': 'Recommend {} hotels in {}. State the name of the hotel and nothing else.', 'top1': 'Recommend one hotel in {}. State the name of the hotel and nothing else.'}


In [7]:
uni_top1 = 'What is the best University in UK. State the name of the University and nothing else.'
uni_topk = 'Recommend {} Universities in the UK. State the name of the University and nothing else.'
uni_pariwise = 'Which University do you recommend more: {} or {}. Simply write the name of the University and nothing else.'

In [99]:
uni_target_words = 'Oxford Cambridge Imperial Warwick Edinburgh Manchester Bristol King’s'.split(' ') + [['University college London', 'UCL'], ['LSE', 'London School of Economics', 'LondonSchoolofEconomicsandPoliticalScience']] 
uni_pairwise_words = 'Oxford Cambridge Imperial Warwick Edinburgh Bristol King’s UCL LSE'.split(' ') 

In [15]:
print(uni_pairwise_words)

['Oxford', 'Cambridge', 'Imperial', 'Warwick', 'Edinburgh', 'Bristol', 'King’s', 'UCL', 'LSE']


## Making functions. 

In [23]:
def load_all(top1_3_5_pickle_id, top1_prompt, topk_prompt, times):
    cf = SimilarityTests()
    lm = LMEO()
        
    top3_prompt = topk_prompt.format('three')
    top5_prompt = topk_prompt.format('five')

    top1 = cf.order_top_1(top1_3_5_pickle_id, top1_prompt, 1000)
    
    top3 = lm.order_set(top3_prompt, top1_3_5_pickle_id, times, 3, prefix='top_3')

    top5 = lm.order_set(top5_prompt, top1_3_5_pickle_id, times, 5, prefix='top_5')

    return top1, top3, top5

def calculate_params(top1, top3, top5, target_words, valpha=None):
    print('Rember to have 0 at beginning of valid alphas')

    valid_alphas = [0] + [10**-i for i in range(1, 5)] if valpha is None else valpha
    print('Begining top3')

    for alpha in valid_alphas:
        try:
            top3_params = st.topk_ranks_to_params(top3, target_words, alpha=alpha)
            break
        except RuntimeError:
            print("Recursion Error for top3. When using alpha: ", alpha)
    else:
        print('No valid alpha found for top3')
        top3_params = None

    print('Begining top5')
    for alpha in valid_alphas:
        try:
            top5_params = st.topk_ranks_to_params(top5, target_words, alpha=alpha)
            break
        except RuntimeError:
            print("Recursion Error for top5. When using alpha: ", alpha)
    else:
        print('No valid alpha found for top5')
        top5_params = None
    
    print('Begining top1')
    for alpha in valid_alphas:
        try:
            top1_params = st.top1_ranks_to_params(top1, target_words, alpha=alpha)
            break
        except RuntimeError:
            print("Recursion Error for top1. When using alpha: ", alpha)
    else:
        print('No valid alpha found for top1')
        top1_params = None

    param_dict = {'top1': top1_params, 'top3': top3_params, 'top5': top5_params}

    num_to_id, id_to_num = st.get_conversions(target_words)

    top3_ordering = [id_to_num[x[1]] for x in top3_params]
    top5_ordering = [id_to_num[x[1]] for x in top5_params]
    top1_ordering = [id_to_num[x[1]] for x in top1_params]

    ordering_dict = {'top3': top3_ordering, 'top5': top5_ordering, 'top1': top1_ordering}

    return param_dict, ordering_dict, num_to_id
                
def do_everything(top1_3_5_pickle_id, top1_prompt, topk_prompt, target_words, times, valpha = None):
    top1, top3, top5 = load_all(top1_3_5_pickle_id, top1_prompt, topk_prompt, times)
    param_dict, ordering_dict, num_to_id = calculate_params(top1, top3, top5, target_words, valpha=valpha)
    return param_dict, ordering_dict, num_to_id

In [31]:
pickle_id = 'Universities1000'
param_dict, ordering_dict, num_to_id = do_everything(pickle_id, uni_top1, uni_topk, uni_target_words, 1000, valpha=[10**-1, 0.5, 1])

Loading Previously Generated Ranks.
Loading Previously Generated Ranks.
Loading Previously Generated Ranks.
Rember to have 0 at beginning of valid alphas
Begining top3
Begining top5
Recursion Error for top5. When using alpha:  0.1
Begining top1


In [32]:
from pprint import pprint
pprint(param_dict)

{'top1': [(7.1447104256538925, 'Oxford'),
          (4.254018821281309, 'OTHER'),
          (2.6133892963692014, 'Cambridge'),
          (-2.001731220472058, 'Imperial'),
          (-2.001731220472058, 'Warwick'),
          (-2.001731220472058, 'Edinburgh'),
          (-2.001731220472058, 'Bristol'),
          (-2.001731220472058, 'King’s'),
          (-2.001731220472058, 'University college London'),
          (-2.001731220472058, 'LSE')],
 'top3': [(3.691491981667278, 'Oxford'),
          (2.0500316416357673, 'Warwick'),
          (2.0500316416357673, 'Bristol'),
          (2.0500316416357673, 'King’s'),
          (2.0500316416357673, 'University college London'),
          (1.8348618696329464, 'Cambridge'),
          (-1.6910796065972815, 'LSE'),
          (-3.823249321962834, 'Edinburgh'),
          (-3.8788896411394225, 'Imperial'),
          (-4.333261848143756, 'OTHER')],
 'top5': [(8.278086480848433, 'Oxford'),
          (6.228428910013009, 'Cambridge'),
          (6.2024844079

In [ ]:
# The alpha has clearly fucked it up some how the ranks for top3 are very wrong.

In [40]:
top1, top3, top5 = load_all(pickle_id,'', '', times=1000)
top5_params = st.topk_ranks_to_params(top5, ['Oxford', 'Cambridge', 'Imperial'], alpha=1)

Loading Previously Generated Ranks.
Loading Previously Generated Ranks.
Loading Previously Generated Ranks.


KeyboardInterrupt: 

In [41]:
with open(r'C:\Users\chris\Documents\LMEO\LMEO\Experiments\pickles\top_3_Universities1000.pkl', 'rb') as f:
    uni_top3_raw = pickle.load(f)

In [42]:
st = SimilarityTests()

conversion = st.rank_to_num(uni_top3_raw, uni_target_words, 1)

In [43]:
print(conversion[1])
print(conversion[0][0:10])

{0: 'Oxford', 1: 'Cambridge', 2: 'Imperial', 3: 'Warwick', 4: 'Edinburgh', 5: 'Bristol', 6: 'King’s', 7: 'University college London', 8: 'LSE', 9: 'OTHER'}
[[0, 1, 2], [0, 1, 2], [1, 0, 9], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2], [0, 1, 2]]


In [65]:
## We will make another simple way to score the ranks. 
## We will just add the position to a score and then choose the min, allocating +1 for any rank which didnt turn up
## Or could -

num_to_id, id_to_num = st.get_conversions(uni_target_words, 1)

formated_ranks = conversion[0]
final_scores_dict = {}

no_appearence_penalty = len(num_to_id)

for num_rank in formated_ranks:
    appeared_unis = set()
    
    for place, uni in enumerate(num_rank):
        chosen_uni = num_to_id[uni]
        
        # Adding the score
        previous_count = final_scores_dict.get(chosen_uni, 0)
        final_scores_dict[chosen_uni] = previous_count + place

        appeared_unis.add(uni)
    
    # Now we will add the penalty for any uni that didnt appear by using the set difference
    not_appeared = set([i for i in range(len(num_to_id))]) - appeared_unis
    for x in not_appeared:
        previous_count = final_scores_dict.get(num_to_id[x], 0)
        final_scores_dict[num_to_id[x]] = previous_count + no_appearence_penalty

sorted_dict = dict(sorted(final_scores_dict.items(), key=lambda item: item[1]))
print(sorted_dict)



{'Oxford': 135, 'Cambridge': 867, 'Imperial': 2611, 'OTHER': 9606, 'Edinburgh': 9782, 'LSE': 9966, 'Warwick': 9990, 'Bristol': 9990, 'King’s': 9990, 'University college London': 9990}


In [133]:
def calculate_sorted_dict(ranks, target_words, other_flag=1):
    st = SimilarityTests()

    ranks = st.rank_to_num(ranks, target_words, other_flag)[0]
    final_scores_dict = {}
    
    num_to_id, _ = st.get_conversions(target_words, 1)
    no_appearence_penalty = len(num_to_id)

    for num_rank in ranks:
        appeared_unis = set()

        for place, uni in enumerate(num_rank):
            chosen_uni = num_to_id[uni]

            # Adding the score
            previous_count = final_scores_dict.get(chosen_uni, 0)
            final_scores_dict[chosen_uni] = previous_count + place

            appeared_unis.add(uni)

        # Now we will add the penalty for any uni that didnt appear by using the set difference
        not_appeared = set(range(len(num_to_id))) - appeared_unis
        for x in not_appeared:
            previous_count = final_scores_dict.get(num_to_id[x], 0)
            final_scores_dict[num_to_id[x]] = previous_count + no_appearence_penalty

    sorted_dict = dict(sorted(final_scores_dict.items(), key=lambda item: item[1]))
    return sorted_dict

with open(r'C:\Users\chris\Documents\LMEO\LMEO\Experiments\pickles\top_5_Universities1000.pkl', 'rb') as f:
    uni_top5_raw = pickle.load(f)

with open(r'C:\Users\chris\Documents\LMEO\LMEO\Experiments\pickles\top_1_Universities1000.pkl', 'rb') as f:
    uni_top1_raw = pickle.load(f)

uni_target_words = 'Oxford Cambridge Imperial Warwick Edinburgh Manchester Bristol King’s'.split(' ') + [['PoliticalScience', 'LSE']] + [['CollegeLondon', 'UCL']]

print(calculate_sorted_dict([x for x in uni_top1_raw if len(x[0]) < 50], uni_target_words))
print(calculate_sorted_dict(uni_top5_raw, uni_target_words))
print(calculate_sorted_dict(uni_top3_raw, uni_target_words))

{'Oxford': 110, 'Cambridge': 9592, 'Imperial': 9702, 'Warwick': 9702, 'Edinburgh': 9702, 'Manchester': 9702, 'Bristol': 9702, 'King’s': 9702, 'PoliticalScience': 9702, 'CollegeLondon': 9702, 'OTHER': 9702}
{'Oxford': 111, 'Cambridge': 889, 'Imperial': 2048, 'PoliticalScience': 3797, 'Edinburgh': 5079, 'Manchester': 9595, 'CollegeLondon': 10700, 'OTHER': 10802, 'Bristol': 10986, 'Warwick': 10993, 'King’s': 11000}
{'Oxford': 135, 'Cambridge': 867, 'Imperial': 2688, 'PoliticalScience': 10539, 'Edinburgh': 10755, 'Manchester': 10980, 'Warwick': 10989, 'Bristol': 10989, 'King’s': 10989, 'CollegeLondon': 10989, 'OTHER': 10989}


In [143]:
freq_results = {"University of Bristol":19525,"Bristol University":5839,"Warwick University":3460,"University of Warwick":13598,"Imperial College London":24265,
"Oxford University":145100,"Cambridge University":104228,"University of Oxford":64776,"University of Cambridge":65050}
freq_results_org = {"University of Bristol":25300,"University of Warwick":16900,"Imperial College London":24265,
"Oxford University":209000,"Cambridge University":169000}
sorted_freq_results = dict(sorted(freq_results_org.items(), key=lambda item: item[1], reverse=True))
print(sorted_freq_results)

{'Oxford University': 209000, 'Cambridge University': 169000, 'University of Bristol': 25300, 'Imperial College London': 24265, 'University of Warwick': 16900}


In [140]:
print(uni_target_words)

ranks, test_con = st.rank_to_num(uni_top5_raw, uni_target_words)
print(test_con)
for i, rank in enumerate(ranks):
    for j, indi in enumerate(rank):
        if indi == 9:
            
            print(repr(uni_top5_raw[i][j]))

['Oxford', 'Cambridge', 'Imperial', 'Warwick', 'Edinburgh', 'Manchester', 'Bristol', 'King’s', ['PoliticalScience', 'LSE'], ['CollegeLondon', 'UCL']]
{0: 'Oxford', 1: 'Cambridge', 2: 'Imperial', 3: 'Warwick', 4: 'Edinburgh', 5: 'Manchester', 6: 'Bristol', 7: 'King’s', 8: 'PoliticalScience', 9: 'CollegeLondon', 10: 'OTHER'}
'UniversityCollegeLondon(UCL)'
'UniversityCollegeLondon'
"King'sCollegeLondon"
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon(UCL)'
'UniversityCollegeLondon'
'UniversityCollegeLondon(UCL)'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon(UCL)'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'UniversityCollegeLondon'
'Universi

In [139]:
'LSE' in 'LondonSchoolofEconomicsandPoliticalScience(LSE)'

def hotel_target_flag(hotel, target_word):
    if isinstance(target_word, list):
        return any((word in hotel) for word in target_word)
    else:
        return target_word in hotel
    
print(any(hotel_target_flag('UniversityCollegeLondon', word) for word in uni_target_words))

True


In [79]:
print(len('There is no definitive answer to this question as it is subjective and depends on individual criteria for what makes a university the "best."'))

141


## Going to test whether this score function is more consistent for hotels.

In [145]:
def temp_load_raw(pickle_id):
    with open(fr'C:\Users\chris\Documents\LMEO\LMEO\Experiments\pickles\top_5_{pickle_id}.pkl', 'rb') as f:
        top5_raw = pickle.load(f)

    with open(fr'C:\Users\chris\Documents\LMEO\LMEO\Experiments\pickles\top_3_{pickle_id}.pkl', 'rb') as f:
        top3_raw = pickle.load(f)

    with open(fr'C:\Users\chris\Documents\LMEO\LMEO\Experiments\pickles\top_1_{pickle_id}.pkl', 'rb') as f:
        top1_raw = pickle.load(f)
    
    return top1_raw, top3_raw, top5_raw

def calculate_all(pickle_id, target_words):
    top1_raw, top3_raw, top5_raw = temp_load_raw(pickle_id)
    top1 = calculate_sorted_dict([x for x in top1_raw if len(x[0]) < 50], target_words)
    top3 = calculate_sorted_dict(top3_raw, target_words)
    top5 = calculate_sorted_dict(top5_raw, target_words)
    return top1, top3, top5

In [144]:
# Getting the target words.
st = SimilarityTests()
hotel_word_dict = st.get_words_dict()

Manchester

In [148]:
birmingham_words = hotel_word_dict['Birmingham']
a, b, c = calculate_all('Birmingham1000', birmingham_words)
print(a)
print(b)
print(c)

{'Hyatt': 9840, 'almaison': 11355, 'dgbaston': 12525, 'OTHER': 13230, 'Vin': 14370, 'Grand': 14595, 'Marriot': 14715, 'Metro': 14730, 'Hampton': 14745, 'Indigo': 14910, 'Staying': 14985, 'ACHotel': 15000, 'adisson': 15000, 'BirminghamCityCentre': 15000, 'Cube': 15000}
{'Hyatt': 1606, 'almaison': 2868, 'Vin': 7104, 'dgbaston': 11974, 'Hampton': 12679, 'OTHER': 13368, 'Staying': 13406, 'ACHotel': 13946, 'Metro': 13980, 'Indigo': 14028, 'adisson': 14054, 'BirminghamCityCentre': 14099, 'Marriot': 14152, 'Grand': 14228, 'Cube': 14401}
{'Hyatt': 1220, 'almaison': 1779, 'Vin': 3425, 'dgbaston': 8390, 'Hampton': 9007, 'ACHotel': 11584, 'OTHER': 11687, 'Staying': 12118, 'adisson': 13086, 'Indigo': 13898, 'Marriot': 14273, 'Metro': 14296, 'BirminghamCityCentre': 14319, 'Grand': 14367, 'Cube': 14626}


In [149]:
bristol_words = hotel_word_dict['Bristol']
bristol_top1, bristol_top3, bristol_top5 = calculate_all('Bristol1000', bristol_words)
print("Top 1:", bristol_top1)
print("Top 3:", bristol_top3)
print("Top 5:", bristol_top5)

Top 1: {'OTHER': 5040, 'Radisson': 10584, 'Marriott': 13314, 'arbour': 13580, 'Hilton': 13776, 'Mercure': 13832, 'Berkeley': 13902, 'Gainsborough': 13986, 'ibis': 13986, 'Avon': 14000, 'DoubleTree': 14000, 'Future': 14000, 'ztec': 14000, 'gabel': 14000}
Top 3: {'OTHER': 3109, 'Radisson': 6121, 'Mercure': 6521, 'Marriott': 8549, 'arbour': 10591, 'Hilton': 13134, 'Berkeley': 13577, 'Future': 13605, 'Avon': 13715, 'Gainsborough': 13792, 'ztec': 13884, 'DoubleTree': 13944, 'ibis': 13944, 'gabel': 13944}
Top 5: {'Mercure': 2659, 'Radisson': 2718, 'OTHER': 3109, 'Marriott': 7390, 'arbour': 9333, 'Hilton': 10604, 'Future': 11332, 'Berkeley': 13226, 'Avon': 13325, 'ztec': 13496, 'Gainsborough': 13857, 'ibis': 13893, 'DoubleTree': 13958, 'gabel': 13958}


In [150]:
newcastle_words = hotel_word_dict['Newcastle']
newcastle_top1, newcastle_top3, newcastle_top5 = calculate_all('Newcastle1000', newcastle_words)
print("Top 1:", newcastle_top1)
print("Top 3:", newcastle_top3)
print("Top 5:", newcastle_top5)

Top 1: {'Malmaison': 9948, 'OTHER': 10608, 'Crowne': 11364, 'Jury': 11748, 'Vin': 11844, 'Grey': 11880, 'County': 11964, 'Sandman': 11964, 'Indigo': 11964, 'Hilton': 11964, 'Dene': 16488}
Top 3: {'Malmaison': 1544, 'Vin': 4429, 'OTHER': 7771, 'Grey': 8121, 'Crowne': 9264, 'Jury': 9935, 'Indigo': 10176, 'County': 10335, 'Hilton': 10360, 'Sandman': 10390, 'Dene': 14869}
Top 5: {'Malmaison': 1038, 'Vin': 2087, 'OTHER': 4586, 'Grey': 5915, 'Crowne': 6512, 'Jury': 9564, 'County': 10130, 'Sandman': 10196, 'Indigo': 10196, 'Hilton': 10411, 'Dene': 14469}
